# NansenCloud - GeoDjango framework for Satellite Data Management


#### First of all we need to initialize Django to work. Let's do some 'magic'

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import os, sys
print sys.path.append('/home/vagrant/py/nansat/')
print sys.path.append('/home/vagrant/py/nansen-cloud/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "nansencloud.settings")
import django
django.setup()

from cat.models import Image

%whos

#### Now we can use the model Image to get info about available data

In [ ]:
# find all images
images = Image.objects.all()
print images

In [ ]:
# get some information about the first image 
img0 = images[0]
print img0.sourcefile.name
print img0.sourcefile.path.address
print img0.sensor.name
print img0.satellite.name
print img0.border

## Structure of NansenCloud
### 1. Cat (app for catalog)
#### 1.1 Image
Any image in NansenCloud
* sourcefile : SourceFile
* status : Status
* sensor : Sensor
* satellite : Satellite
* start_date : DateTimeField
* stop_date : DateTimeField
* mapper = CharField
* border = PolygonField

#### 1.2 SourceFile
File with original data
* name : CharField
* path : Location
* urls : Location(s)

#### 1.3 Location
Path of the file
* address : CharField
* type : [FS, FTP, HTTP, OpenDAP, SSH]

#### 1.4 Sensor
Name of the sensor
* name : CharField

#### 1.5 Satellite
Name of the satellite
* name : CharField

#### 1.6 Status
Bad or good status of opening
* status : [BAD_STATUS, GOOD_STATUS]

#### 1.7 Search
Web-search criteria
* sdate : DateTimeField
* date0 : DateTimeField
* date1 : DateTimeField
* status : Status
* satellite : Satellite
* sensor : Sensor
* polygon : PolygonField

### 2. Proc (app for operational processing)

#### 2.1 MERISWeb
Processing of Meris data for WEB
* image : Image
* resolution : CharField
* level : IntegerField
* product : Product
* daily : BooleanField
* chain : Chain

#### 2.2 SARWeb
Processing of Meris data for WEB
* polarizations : [Polarizations]
* quicklooks : [Products]
* chain : Chain

#### 2.3 Chain
Operational processing parameters
* name : CharField
* webname : CharField
* description : CharField

#### 2.4 Product
Online product
* image : Image
* standard_name
* long_name
* minimum
* maximum

## More advanced usage of NansenCloud: search for images

In [ ]:
# Search for image taken later than 3 March 2012
#     double underscore (__) is used to join operators
#     gte means 'greater than or equal'
im = Image.objects.filter(start_date__gte='2012-03-03 00:00')
print im

In [ ]:
# Search for images overalapping a polygon
from django.contrib.gis.geos import Polygon
# first, create a polygon for searching
p = Polygon(((60, 5), (90, 5), (90, 25), (60, 25), (60, 5)))
# perform spatial search 
im = Image.objects.filter(border__intersects=p)[0]
print im

### Finally, get data

In [ ]:
n = im.get_nansat()
n.resize(0.1)
sigma0_water_HH = n['sigma0_water_HH']

In [ ]:
import numpy as np
plt.figure(figsize=(15,15))
plt.imshow(sigma0_water_HH, vmin=0, vmax=0.08, cmap='gray')
plt.colorbar()
plt.show()